# New model design

Here is the design of a word to vec Embeding to find the polarity of a critic.

In [1]:
import logging
import mlflow
import polars as pl
import numpy as np
import datetime

logger = logging.getLogger("mlflow")
logger.setLevel(logging.DEBUG)

In [2]:
df_train = (pl.scan_csv('../data/train/exp_train.csv')).collect()
df_test = (pl.scan_csv('../data/test/exp_test.csv')).collect()
df_val = (pl.scan_csv('../data/valid/exp_valid.csv')).collect()
df_train.head()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,…,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999,polarity
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

X_train = df_train.select(pl.exclude("polarity")).to_numpy()
y_train = df_train.select("polarity").to_numpy().ravel()

X_test = df_test.select(pl.exclude("polarity")).to_numpy()
y_test = df_test.select("polarity").to_numpy().ravel()

In [29]:
dataset_preview = df_train.head()

dataset_preview.write_csv('../data/dataset_preview.csv')

In [4]:
def compute_metrics(pred, true):
    # pred et true doivent être des tensors 0/1
    if type(pred) != np.ndarray:
        pred = pred.int()
        true = true.int()

    TP = ((pred == 1) & (true == 1)).sum().item()
    TN = ((pred == 0) & (true == 0)).sum().item()
    FP = ((pred == 1) & (true == 0)).sum().item()
    FN = ((pred == 0) & (true == 1)).sum().item()

    accuracy  = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0

    return accuracy, precision, recall


In [5]:
desc = "a sklearn logistic regression model for predict polarity \n the dataset had been preprosed with tfidf vectorizer with : \n - max_features=5000\n - strip_accents='ascii'\n - lowercase=True\n - stop_words=list(fr_stop)\n - spacy.lang.fr.stop_words stop words"

In [6]:
import git

repo = git.Repo(".", search_parent_directories=True)

remote = repo.remotes.origin.url
remote_clean = remote.replace(".git", "")

git_user = remote_clean.split("/")[-2]
git_repo = remote_clean.split("/")[-1]
git_branch = repo.active_branch.name
git_commit = repo.head.commit.hexsha

In [12]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("logistic_regression_experiment")
mlflow.sklearn.autolog()
with mlflow.start_run():

    # training documentations
    hyperParam = {
        'C':1.25,
        'pnalty':'l2',
        'solver':'saga'
    }
    mlflow.log_params(hyperParam)
    mlflow.set_tags({
        "mlflow.user": git_user,
        "git.repo": git_repo,
        "git.remote": remote,
        "mlflow.source.git.branch": git_branch,
        "mlflow.source.git.commit": git_commit,
        "mlflow.source.name":"notebook/model_design2.ipynb",
        "mlflow.source.type":"NOTEBOOK",
        "mlflow.note.content":desc
    })
    mlflow.log_artifact("../data/dataset_preview.csv", artifact_path="dataset")

    # trraining
    LR = LogisticRegression()
    LR.fit(X_train, y_train)
    preds = LR.predict(X_test)

    # testing
    acc, prec, rec = compute_metrics(y_test, preds)
    date = datetime.datetime.now()
    
    # Confusion matrix
    cm = confusion_matrix(y_test, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax)
    ax.set_title("Confusion Matrix")
    mlflow.log_figure(fig, "test_confusion_matrix.png")
    plt.close(fig)

    # ROC AUC
    probs = LR.predict_proba(X_test)[:, 1]  # binary example
    auc = roc_auc_score(y_test, probs)
    fpr, tpr, _ = roc_curve(y_test, probs)
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr)
    ax.set_xlabel("FPR")
    ax.set_ylabel("TPR")
    ax.set_title(f"ROC curve (AUC = {auc:.3f})")
    mlflow.log_figure(fig, "test_roc_curve.png")
    plt.close(fig)
    mlflow.log_metrics({
        "testing_accuracy": acc,
        "testing_precision": prec,
        "testing_recall": rec,
        "testing_auc": auc
        })
    mlflow.sklearn.log_model(sk_model=LR, name=f"polarity_logistic_regression_{datetime.datetime.now().microsecond}")

2025/11/29 18:21:24 DEBUG mlflow.utils.autologging_utils: Called autolog() method for sklearn autologging with args '()' and kwargs '{'log_input_examples': False, 'log_model_signatures': True, 'log_models': True, 'log_datasets': True, 'disable': False, 'exclusive': False, 'disable_for_unsupported_versions': False, 'silent': False, 'max_tuning_runs': 5, 'log_post_training_metrics': True, 'serialization_format': 'cloudpickle', 'registered_model_name': None, 'pos_label': None, 'extra_tags': None}'
2025/11/29 18:21:25 DEBUG mlflow.utils.gorilla: Patch fn on destination already existed. Overwrite old patch.
2025/11/29 18:21:25 DEBUG mlflow.utils.gorilla: Patch fn on destination already existed. Overwrite old patch.
2025/11/29 18:21:25 DEBUG mlflow.utils.gorilla: Patch fn on destination already existed. Overwrite old patch.
2025/11/29 18:21:25 DEBUG mlflow.utils.gorilla: Patch fn on destination already existed. Overwrite old patch.
2025/11/29 18:21:25 DEBUG mlflow.utils.gorilla: Patch fn on 

🏃 View run indecisive-bat-802 at: http://localhost:5000/#/experiments/1/runs/7bb692c769bc48648d6a0e76778b5fb8
🧪 View experiment at: http://localhost:5000/#/experiments/1
